In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import morphology
from skimage.measure import label

In [2]:
def neighbours4(y, x):
  return (y,x-1),(y,x+1),(y-1,x),(y+1,x)


def get_boundaries(LB,label):
  pxs = np.where(LB == label)
  boundaries = []
  for y,x in zip(*pxs):
    for yn, xn in neighbours4(y,x):
      if yn < 0 or yn > LB.shape[0] - 1:
        boundaries.append((y,x))
        break
      elif xn < 0 or xn > LB.shape[1] - 1:
        boundaries.append((y,x))
        break
      elif LB[yn,xn] != label:
        boundaries.append((y,x))    
        break
  return boundaries

def draw_boundaries(LB,label):
  BB = np.zeros_like(LB)
  pos = np.where(LB == label)
  BB[pos] = LB[pos]

  boundaries = get_boundaries(BB,label)
  for y, x in boundaries:
    BB[y,x] = label + 1
  return BB

In [3]:
file = np.load('ps.npy')

In [4]:
labeled = label(file)

In [5]:
print(np.max(labeled))

500


In [6]:
def get_boundary_path(image,label):
  boundaries = get_boundaries(image,label)
  moves = [(-1,0),(-1,1),(0,1),(1,1),(1,0),(1,-1),(0,-1),(-1,-1)]
  path = []
  coords = boundaries[0]
  while len(boundaries) > 0:
    move_candidates = []
    new_coords = ()
    for move in moves:
      new_coords = (coords[0]+move[0],coords[1]+move[1])
      if new_coords in boundaries:
        move_candidates.append(True)
      else:
        move_candidates.append(False)
    
    if not move_candidates[-1] and move_candidates[0]:
      coords = (coords[0]+moves[0][0],coords[1]+moves[0][1])
      path.append(0)
    else:
      for i in range(7):
        if not move_candidates[i] and move_candidates[i+1]:
          coords = (coords[0]+moves[i+1][0],coords[1]+moves[i+1][1])
          path.append(i+1)
          break

    boundaries.remove(coords)
  return sorted(path)

In [25]:
image = file.copy()
total = np.max(labeled)
types = {}
found = 0

masks = [np.ones((4,6))]
result = morphology.binary_opening(image,masks[0]).astype('int')
labeled_c = label(result)
amount = np.max(labeled_c)
found = amount
types[0] = amount

new_type = 1
for i in range(1,total+1):
  pxs = np.where(labeled == i)
  if len(pxs[0])>0:
    labeled_n = np.zeros_like(labeled)
    labeled_n[pxs] = 1
    cropped = labeled_n[min(pxs[0]):max(pxs[0])+1,min(pxs[1]):max(pxs[1])+1]

    already_counted = False
    for j in range(0,len(masks)):
      if cropped.shape == masks[j].shape:
        if (cropped == masks[j]).all():
          already_counted = True
    
    if already_counted:
      continue

    masks.append(cropped)
    result = morphology.binary_opening(image,cropped).astype('int')
    labeled_c = label(result)
    amount = np.max(labeled_c)
    
    if cropped.shape == masks[0].shape:
      if masks[0].sum() - (cropped == masks[0]).sum() == 4:
        amount -= types[0]

    types[new_type] = amount
    new_type += 1

    found+=amount
    if found==total:
      break

In [28]:
print(types)

for t in range(len(types)):
  print('Mask: \n', masks[t])
  print('Amount: ',types[t])

{0: 92, 1: 94, 2: 123, 3: 95, 4: 96}
Mask: 
 [[1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1.]]
Amount:  92
Mask: 
 [[1 1 1 1]
 [1 1 1 1]
 [1 1 0 0]
 [1 1 0 0]
 [1 1 1 1]
 [1 1 1 1]]
Amount:  94
Mask: 
 [[1 1 1 1]
 [1 1 1 1]
 [0 0 1 1]
 [0 0 1 1]
 [1 1 1 1]
 [1 1 1 1]]
Amount:  123
Mask: 
 [[1 1 0 0 1 1]
 [1 1 0 0 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]]
Amount:  95
Mask: 
 [[1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 0 0 1 1]
 [1 1 0 0 1 1]]
Amount:  96
